# 建立推薦程式(用CV推薦職缺)，使用訓練好的模型分類，計算相似度

程式步驟 (使用 scikit-learn)

> 1. 使用者輸入CV，以及工作條件

> 2. 清洗CV，轉換成BOW， 丟入訓練模型，顯示預測分類結果。

> 3. 利用預測分類結果，與使用者輸入工作條件，從mongoDB撈取職缺

> 4. 將職缺轉換成BOW

> 5. 計算CV與職缺的相似度，推薦相似度最近的10筆工作


效果展示

> 測試CV內容

    '''最近工作\n\n公司：\tXX醫院\t行業：\t醫療/護理/衛生\n職位：\t醫生\n最高學歷\t\n
    最高學歷\n\n學校：\t中醫藥大學\n學歷：\t本科\t專業：\t\n醫藥學\n工作經驗\t\n工作經驗\n\n公司：\t\n
    XX醫院\n2012/7–2017/7\n職位：\t醫生\n行業：\t醫療/護理/衛生\n部門：\t醫藥部\n工作內容：\n1.
    了解各種儀器設備的使用方法。2.參與手術工作，鍛煉手術操作能力。3.熟悉實際操作中所出現的問題並通過各種
    方法避免和克服。4.勤學好問，大膽展示自我，學會了要禮貌待人，要踏實幹事，要提高個人綜合素質。\n教育經歷\t\n
    教育經歷\n\n學校：\t\n中醫藥大學\n2007/9–2011/6\n專業：\t醫藥學\t本科\t\n自我評價\t\n自我評價\n\n在校期
    間，學習了解剖學、生物化學、生理學、病理學、精神學等等課程，在校成績優異，擔任學生幹部的職務，曾多次獲得過
    學校獎學金，平時積極主動參加校內活動，曾負責在專業內組織過醫學演講。學習態度端正，能夠主動學習，認真對待每
    一次學習的機會，我希望在學習理論知識的同時，能夠增強自己的實踐經驗，我 相信沒有做不到，只有不想做。\n求職
    意向\t\n求職意向\n\n到崗時間：\t一個月之內\n工作性質：\t全職\n希望行業：\t醫療/護理/衛生\n目標地點：\t
    北京\n期望月薪：\t面議/月\n目標職能：\t醫生\n語言能力\t\n語言能力\n\n英語：\t\n良好\n\n聽說：\t\n良好
    \n\n讀寫：\t\n良好\n\n證書\t\n證書\n\n大學英語四級'''

> 推薦結果

    1. 衛教護理師-兼職 (北區) 康樂保股份有限公司
    ====================
    2. 門市業務人員(台北市) 維康醫療用品 _美德耐股份有限公司
    ====================
    3. 亞東紀念醫院-巡迴健檢護理師 長德醫院管理顧問有限公司
    ====================
    4. 照顧服務員 財團法人台灣省私立健順養護中心


## 1. Clean User input CV

模擬用戶輸入數據

In [63]:
# User input data

test_cv = '''最近工作\n\n公司：\tXX醫院\t行業：\t醫療/護理/衛生\n職位：\t醫生\n最高學歷\t\n
最高學歷\n\n學校：\t中醫藥大學\n學歷：\t本科\t專業：\t\n醫藥學\n工作經驗\t\n工作經驗\n\n公司：\t\n
XX醫院\n2012/7–2017/7\n職位：\t醫生\n行業：\t醫療/護理/衛生\n部門：\t醫藥部\n工作內容：\n1.
了解各種儀器設備的使用方法。2.參與手術工作，鍛煉手術操作能力。3.熟悉實際操作中所出現的問題並通過各種
方法避免和克服。4.勤學好問，大膽展示自我，學會了要禮貌待人，要踏實幹事，要提高個人綜合素質。\n教育經歷\t\n
教育經歷\n\n學校：\t\n中醫藥大學\n2007/9–2011/6\n專業：\t醫藥學\t本科\t\n自我評價\t\n自我評價\n\n在校期
間，學習了解剖學、生物化學、生理學、病理學、精神學等等課程，在校成績優異，擔任學生幹部的職務，曾多次獲得過
學校獎學金，平時積極主動參加校內活動，曾負責在專業內組織過醫學演講。學習態度端正，能夠主動學習，認真對待每
一次學習的機會，我希望在學習理論知識的同時，能夠增強自己的實踐經驗，我 相信沒有做不到，只有不想做。\n求職
意向\t\n求職意向\n\n到崗時間：\t一個月之內\n工作性質：\t全職\n希望行業：\t醫療/護理/衛生\n目標地點：\t
北京\n期望月薪：\t面議/月\n目標職能：\t醫生\n語言能力\t\n語言能力\n\n英語：\t\n良好\n\n聽說：\t\n良好
\n\n讀寫：\t\n良好\n\n證書\t\n證書\n\n大學英語四級'''

test_work_area = '台北市'
test_work_exp = 6
test_edu = '大學'


定義清洗函數

In [21]:
import jieba
import re

In [22]:
# data cleaning
def string_clean(CV_data): 
    '''
    input: string (original CV description)
    output: string (clean CV description)
    '''
    job_desc = CV_data.split('\n') # 根據換行符號轉乘 List格式
    job_words = ''
    
    for words in job_desc:
        words = words.replace('\t', ' ').replace('\r', ' ')
        words = re.sub(r'[^\w\s]',' ', words)  # remove all punctuations
        words = re.sub(r'\d+', ' ', words)  # remove all numbers
        words = words.strip() # remove white space
        words += ' '
        job_words += words
        
    return (job_words)

In [23]:
jieba.load_userdict('jieba_data/Jobcontent_dict.txt') # 指定辭典檔

In [24]:
# 排除字元表單 stopword, 開啟 'Jobcontent_stopwords.txt'檔案

with open(file='jieba_data/Jobcontent_stopwords.txt',mode='r', encoding="UTF-8") as file:
    stop_words = file.read().split('\n')
    stop_words = [i.strip() for i in stop_words]

In [25]:
def isEnglish(s): # 檢查字元是否為英文
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def jieba_cut(data, stop_words):
    
    # 使用結巴斷詞
    seg_result = jieba.cut(data, cut_all=False)

    # 篩選斷詞，去掉單一中文字
    lst_seg = []
    
    for i in list(seg_result):
        i = i.strip()
        if len(i) < 1:  #排除空值
            continue
        elif i in stop_words: # 排除stopwords
            continue
        elif isEnglish(i) == False and len(i) == 1: #排除單一中文字
            continue
        else:
            lst_seg.append(i)
   
    return lst_seg

In [38]:
test_cv_clean = string_clean(test_cv)
test_cv_clean = jieba_cut(test_cv_clean, stop_words)
test_cv_clean = [' '.join(test_cv_clean)]

In [39]:
test_cv_clean

['XX 醫院 行業 醫療 護理 衛生 職位 醫生 學歷 學歷 學校 醫藥 大學 學歷 本科 專業 醫藥 XX 醫院 職位 醫生 行業 醫療 護理 衛生 部門 醫藥 儀器 設備 方法 手術 鍛煉 手術 操作能力 操作 通過 方法 避免 克服 勤學 好問 展示 幹事 教育 經歷 教育 經歷 學校 醫藥 大學 專業 醫藥 本科 評價 評價 解剖 生物 化學 病理 課程 成績 擔任 學生 幹部 職務 獲得過 學校 參加校 專業 組織 醫學 演講 對待 理論 知識 增強 到崗 全職 行業 醫療 護理 衛生 期望 醫生 語言 語言 英語 讀寫 證書 證書 大學 英語']

## 2. Transform data, and predict the classification

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from joblib import dump, load
import pickle

In [41]:
# The scikit-learn version is 0.22.1.

In [42]:
def cv_category_predict(cv_data):

    vectorizer = pickle.load(open("model/vectorizer.pickel", "rb")) 
    transform_content = vectorizer.transform(cv_data) # CountVectorizer transform
    X_test = transform_content.toarray()
    # vectorizer.inverse_transform(X_test)

    model = load('model/MultinomialNB.joblib') 
    y_pred = model.predict(X_test)  # MultinomialNB transform

    le = pickle.load(open("model/le.pickel", "rb"))
    y_pred_category = le.inverse_transform(y_pred)
    y_pred_category = y_pred_category[0]
    
    return y_pred_category

In [44]:
test_cv_category = cv_category_predict(test_cv_clean)

顯示預測結果

In [64]:
print('CV data after jieba: ', test_cv_clean) 
print('='*20)
print('Predicted category: ', test_cv_category)

CV data after jieba:  ['XX 醫院 行業 醫療 護理 衛生 職位 醫生 學歷 學歷 學校 醫藥 大學 學歷 本科 專業 醫藥 XX 醫院 職位 醫生 行業 醫療 護理 衛生 部門 醫藥 儀器 設備 方法 手術 鍛煉 手術 操作能力 操作 通過 方法 避免 克服 勤學 好問 展示 幹事 教育 經歷 教育 經歷 學校 醫藥 大學 專業 醫藥 本科 評價 評價 解剖 生物 化學 病理 課程 成績 擔任 學生 幹部 職務 獲得過 學校 參加校 專業 組織 醫學 演講 對待 理論 知識 增強 到崗 全職 行業 醫療 護理 衛生 期望 醫生 語言 語言 英語 讀寫 證書 證書 大學 英語']
Predicted category:  醫療╱保健服務類



## 3. 連線至職缺，利用條件篩選

#### 使用 mongoDB 連線篩選職缺

In [69]:
import pymongo
import pandas as pd
from secrets import HOST

In [103]:
def mongo_select_jobs(category, area = None,  workExp = None, edu = None): 
    
    global mycol
    client = pymongo.MongoClient(HOST) 

    db = client['final_104'] 
    mycol = db['final_data'] 

    
    filter_dict = {'jobCat_main': category} # create filter for mongoDB
    
    if area != None:  
        filter_dict['Work_area_clean'] = area # filter data by area

    data = mycol.find(filter_dict,{'_id':0}) # get data from mongoDB
    output = [i for i in data]
    
    # turn output into dataframe
    df = pd.DataFrame(output)
    
     # filter data by work experience
    if workExp is not None:
        lst_workExp = list(i for i in range(0, workExp+1))
        df = df[df['workExp_clean'].astype(int).isin(lst_workExp)]    
    
    # filter data by education status
    if edu is not None:
        df = df[df['Edu_clean'].str.contains("大學")]

    filter_result = df.values.tolist()

    # 將資料儲存成flask可用的dictionary格式

    dict_data = {} 

    for i in filter_result:
        jobURL, Job_Name, Company, Job_Description, concate_jieba = i[0], i[1], i[2], i[8], i[9]
        dict_data[jobURL] = { 
        'Job_Name': Job_Name,
        'Company': Company,
        'Job_Description': Job_Description,
        'concate_jieba': concate_jieba }
        
    return dict_data    

In [104]:
# 利用預測分類結果，與使用者輸入工作條件，從mongoDB撈取職缺
jobs_query = mongo_select_jobs(test_cv_category, test_work_area, test_work_exp, test_edu)
print('總共有幾筆資料: ' , len(jobs_query))

總共有幾筆資料:  921


## 4. 將結巴後的職缺轉換成BOW

In [106]:
 # 將結巴後的斷詞，利用CountVectorizer轉換減少字詞數量
def turn_content_BOW(content):
    X_content = [ " ".join(content)]
    vectorizer = pickle.load(open("model/vectorizer.pickel", "rb")) 
    transform_content = vectorizer.transform(X_content) 
    X_content_array = transform_content.toarray()
    return X_content_array

## 5. 計算相似度，使用 cosine similarity

In [107]:
from sklearn.metrics.pairwise import cosine_similarity

In [108]:
def compute_similarity(cv_BOW, job_id, job_jieba_data):
    
    job_BOW = turn_content_BOW(job_jieba_data)
    job_prob = cosine_similarity(cv_BOW, job_BOW)
    job_prob = round(job_prob.tolist()[0][0], 6)
    
    return job_prob, job_id 

In [122]:
def show_recommendation_result(cv_clean, jobs_query):
    cv_BOW = turn_content_BOW(cv_clean)

    dict_prob_id = {}

    for i,j in jobs_query.items():
        job_jieba_data = j['concate_jieba']
        similarity = compute_similarity(cv_BOW, i, job_jieba_data)
        dict_prob_id[similarity[0]] = similarity[1]
        result = [(k, dict_prob_id[k]) for k in sorted(dict_prob_id.keys(), reverse = True)[0:10]]
    
    list_ten_result = []
    
    for i,j in result:
        data = jobs_query[j]
        lst_result = [i, data['Job_Name'], data['Company'], data['Job_Description'], j]
        list_ten_result.append(lst_result)

    return(list_ten_result)



In [123]:
# show 10 job recommendation to user
final_result = show_recommendation_result(test_cv_clean, jobs_query)
final_result

[[0.365896, '衛教護理師-兼職 (北區)', '康樂保股份有限公司', '病患腸造口術後衛教\n \n具備護理執照', '514gg'],
 [0.355389,
  '門市業務人員(台北市)',
  '維康醫療用品 _美德耐股份有限公司',
  '醫療用品/藥局門市之業務人員需具醫護相關背景醫藥用品銷售，拜訪醫院相關人員支援門市促銷活動具良好溝通協調能力與抗壓性\n \n積極，具服務熱忱醫護相關科系畢或相關經驗尤佳具醫療藥品銷售經驗一年以上者尤佳',
  '6nqky'],
 [0.30933,
  '亞東紀念醫院-巡迴健檢護理師',
  '長德醫院管理顧問有限公司',
  '1、工作區域:巡迴體檢,以北臺灣為主。2、工作內容:負責執行巡迴體檢相關專業工作及相關行政事宜。3、工作時間:無輪夜班、正常工時。4、具護理師或護士證書。5、熟悉醫學中心護理服務程序，樂在溝通協調互動親和。6、善於與人溝通，反應能力快、機動性高、能獨立作業。7、具職業病安全衛生講習結業證書者優先錄取。8、其他主管交付工作及臨時交辦事項。9、底薪5萬以上，實薪面洽10、男女均可11、保障年薪60萬-100萬12、歡迎有相關行業者加入行列,薪資面議\n \n1、可取得個人國家級醫學中心之護理執業履歷。2、對巡迴健檢實務有相當的信心與熱忱。3、個人有相當期許，並對高獎金有強烈企圖心。',
  'zad4'],
 [0.295689,
  '照顧服務員',
  '財團法人台灣省私立健順養護中心',
  '照顧長輩日常生活  護理評估、醫療照護  給藥、家屬溝通 輔療性活動帶領\n \n意者可電子郵件或電話主動和我們連絡',
  '6wzu3'],
 [0.284067,
  '醫美護理諮詢師（台北）',
  '漢蒂妮醫美診所',
  '1.需具備護士/護理師證照。  2.醫美專業諮詢銷售服務、術前術後衛教宣導。 3.有開刀房經驗佳，維護工作環境整潔,器械保養消毒。 4.依照診所標準作業流程執行護理業務,以及整形外科相關庶務。 5.進行靜脈注射,肌肉注射,打點滴 6.提供基本的病患基礎醫療照顧,如傷口包紮.換藥等\n \n1.具備細心.認真.負責任.之正確工作態度錄取條件及工作內容: 2.底薪＋獎金ps.履歷各項資料完整優先考慮(如:學歷、工作經驗、專業證照、自傳)',
  '6jmdf'